# AWS Integration Dashboard - Setup & Validation

This notebook validates your SageMaker environment and sets up the required AWS resources.

## What This Notebook Does:
1. Checks Python and package versions
2. Validates AWS credentials (via SageMaker execution role)
3. Tests connectivity to AWS Bedrock
4. Tests connectivity to AWS Timestream
5. Creates Timestream database and UEReports table
6. Displays configuration summary

**Run this notebook first before using the other notebooks!**

## Step 1: Import Required Modules

In [ ]:
import sys
import os
import boto3
from datetime import datetime

# Add modules directory to path
sys.path.insert(0, os.path.abspath('../modules'))

from bedrock_streaming import BedrockClient
from timestream_client import TimestreamClient

print("✅ All modules imported successfully")
print(f"Python version: {sys.version}")
print(f"boto3 version: {boto3.__version__}")

## Step 2: Configure Environment Variables

In [ ]:
# Configuration
AWS_REGION = os.getenv('AWS_REGION', 'us-east-1')
TIMESTREAM_DATABASE_NAME = os.getenv('TIMESTREAM_DATABASE_NAME', 'SuperAppDB')
TIMESTREAM_TABLE_NAME = 'UEReports'
AWS_ACCOUNT_ID = '012351853258'

print("Configuration:")
print(f"  AWS Region: {AWS_REGION}")
print(f"  Timestream Database: {TIMESTREAM_DATABASE_NAME}")
print(f"  Timestream Table: {TIMESTREAM_TABLE_NAME}")
print(f"  AWS Account: {AWS_ACCOUNT_ID}")

## Step 3: Check AWS Credentials

SageMaker uses an execution role, so you don't need explicit access keys.

In [ ]:
# Get current AWS identity
sts = boto3.client('sts', region_name=AWS_REGION)

try:
    identity = sts.get_caller_identity()
    print("✅ AWS Credentials Valid")
    print(f"  Account: {identity['Account']}")
    print(f"  ARN: {identity['Arn']}")
    print(f"  User ID: {identity['UserId']}")
except Exception as e:
    print(f"❌ AWS Credentials Error: {str(e)}")
    print("\nPlease ensure your SageMaker notebook instance has an IAM role attached.")

## Step 4: Test Bedrock Connectivity

In [ ]:
print("Testing Bedrock connectivity...\n")

bedrock = BedrockClient(region_name=AWS_REGION)
result = bedrock.test_connection()

if result['status'] == 'success':
    print("✅ Bedrock Connection Successful")
    print(f"  Region: {result['region']}")
    print(f"  Model: {result['model_id']}")
    print(f"  Test Response: {result['response']}")
else:
    print("❌ Bedrock Connection Failed")
    print(f"  Error: {result['message']}")
    print("\nPlease ensure your SageMaker execution role has 'bedrock:InvokeModel' permission.")

## Step 5: Setup Timestream Database & Table

In [ ]:
print("Setting up Timestream database...\n")

timestream = TimestreamClient(
    database_name=TIMESTREAM_DATABASE_NAME,
    table_name=TIMESTREAM_TABLE_NAME,
    region_name=AWS_REGION
)

try:
    setup_result = timestream.setup_database()
    print("✅ Timestream Setup Complete")
    print(f"  Database: {setup_result['database']['message']}")
    print(f"  Table: {setup_result['table']['message']}")
except Exception as e:
    print(f"❌ Timestream Setup Failed: {str(e)}")
    print("\nPlease ensure your SageMaker execution role has Timestream permissions.")

## Step 6: Test Timestream Query

In [ ]:
print("Testing Timestream query...\n")

try:
    df = timestream.query()
    print(f"✅ Timestream Query Successful")
    print(f"  Records found: {len(df)}")
    
    if len(df) > 0:
        print("\nSample data:")
        display(df.head())
    else:
        print("\n⚠️  Table is empty. Use write_records() to add sample data.")
        print("   Or proceed to the Claude chat notebook!")
except Exception as e:
    print(f"❌ Timestream Query Failed: {str(e)}")

## Step 7: Configuration Summary

In [ ]:
from IPython.display import Markdown

summary = f"""
## ✅ Setup Complete!

Your SageMaker environment is ready for the AWS Integration Dashboard.

### Configuration:
- **AWS Region:** {AWS_REGION}
- **Timestream Database:** {TIMESTREAM_DATABASE_NAME}
- **Timestream Table:** {TIMESTREAM_TABLE_NAME}
- **Claude Model:** anthropic.claude-3-sonnet-20240229-v1:0

### Next Steps:
1. **Open `01_claude_chat.ipynb`** to try the interactive Claude 3 chat
2. **Open `02_timestream_dashboard.ipynb`** to explore Timestream data

### Timestamp:
{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

display(Markdown(summary))

## Optional: Add Sample Data to Timestream

Uncomment and run this cell to add sample data to the UEReports table:

In [ ]:
# # Uncomment to add sample data
# sample_records = [
#     {
#         'Dimensions': [
#             {'Name': 'device_id', 'Value': 'device-001'},
#             {'Name': 'location', 'Value': 'datacenter-1'}
#         ],
#         'MeasureName': 'cpu_usage',
#         'MeasureValue': '75.5',
#         'MeasureValueType': 'DOUBLE'
#     },
#     {
#         'Dimensions': [
#             {'Name': 'device_id', 'Value': 'device-002'},
#             {'Name': 'location', 'Value': 'datacenter-1'}
#         ],
#         'MeasureName': 'cpu_usage',
#         'MeasureValue': '82.3',
#         'MeasureValueType': 'DOUBLE'
#     }
# ]

# result = timestream.write_records(sample_records)
# print(result['message'])